In [140]:
import pandas as pd

In [141]:
data = pd.read_csv("./data/partition_database.csv")

In [142]:
data['full_name'] = data['family_name'] + " " + data['given_name']

In [143]:
data.drop([
	'author_name', 
	'line', 
	'violence', 
	'harmony', 
	'conflict', 
	'vitality', 
	'cause_of_death', 
	'non-human', 
	'char_story_title', 
	'wkt', 
	'gender_author',
	# 'start_page_event',
	'temporality', 
	# 'text_title', 
	'interaction', 
	'narrative_status',
	'specific_interaction',
	'nickname', 
	'location_modality',
	'presentmentioned', 
	'ontology', 
	'collection_id',
	'collection_recordtitle',
	'birth_date',
	'end_page_event',
	# 'page_event_number',
	'gender',
	'religious_conversion',
	'mentioned'
], inplace=True, axis=1)

In [145]:
print(data.iloc[0])

text_title                                          Train to Pakistan
event_id                                                          126
start_page_event                                                   12
page_event_number                                                   1
text_place_title                    Sutlej River in Train to Pakistan
character_id                                                    112.0
family_name                                                     Singh
given_name                                                     Juggut
page_first_mentioned                                              9.0
number                                                         Single
religion                                                         Sikh
relig_att_char                                            Speculation
country_of_birth                                                  NaN
place_name                                               Sutlej River
location_religion   

In [146]:
data['event_order'] = data['start_page_event'] + data['page_event_number'] / 100

In [147]:
data[data['character_id'] == 112]['location_name']

0       Mano Majra
580            NaN
582            NaN
585            NaN
590            NaN
607            NaN
609            NaN
614            NaN
618            NaN
629            NaN
666            NaN
667            NaN
669     Mano Majra
670     Mano Majra
1205           NaN
1220           NaN
1271           NaN
1467           NaN
1476           NaN
1485           NaN
1497           NaN
1951           NaN
1960           NaN
1963           NaN
4142           NaN
4191           NaN
4213           NaN
4222           NaN
4226           NaN
4310           NaN
4324           NaN
4335           NaN
4337           NaN
4388           NaN
4415           NaN
4422           NaN
4494           NaN
4495           NaN
Name: location_name, dtype: object

In [148]:
# data[(data[data['character_id'] == 112]) and (data['event_order'] == min(data[data['character_id'] == 112]['event_order']))]
_df = data[data['character_id'] == 112]
_df = _df.dropna(subset=['location_country'])
_df[_df['event_order'] == min(_df['event_order'])]['location_country'][0]

'India'

In [162]:
data.dropna(subset=['location_country', 'location_name', 'character_id'], inplace=True)

obj_start_country = {}
obj_start_place = {}
obj_end_country = {}
obj_end_place = {}

_df = data.groupby(by=data['character_id'].tolist())
for char_id, event_order in _df['event_order'].min().iteritems():
	_df_char = data[data['character_id'] == char_id]
	# print(char_id, _df_char[_df_char['event_order'] == event_order].iloc[0]['location_country'])
	obj_start_country[char_id] = _df_char[_df_char['event_order'] == event_order].iloc[0]['location_country']
	obj_start_place[char_id] = _df_char[_df_char['event_order'] == event_order].iloc[0]['location_name']

for char_id, event_order in _df['event_order'].max().iteritems():
	_df_char = data[data['character_id'] == char_id]
	# print(_df_char[_df_char['event_order'] == event_order].iloc[0]['location_country'])
	obj_end_country[char_id] = _df_char[_df_char['event_order'] == event_order].iloc[0]['location_country']
	obj_end_place[char_id] = _df_char[_df_char['event_order'] == event_order].iloc[0]['location_name']

In [163]:
data['initial_country'] = data.apply(lambda row: obj_start_country[row.character_id], axis = 1)
data['final_country'] = data.apply(lambda row: obj_end_country[row.character_id], axis = 1)

data['initial_place'] = data.apply(lambda row: obj_start_place[row.character_id], axis=1)
data['final_place'] = data.apply(lambda row: obj_end_place[row.character_id], axis = 1)

In [168]:
data.dtypes
data[['text_title', 'event_order', 'full_name', 'initial_country', 'final_country', 'initial_place', 'final_place']]

,text_title,event_order,full_name,initial_country,final_country,initial_place,final_place
0,Train to Pakistan,12.01,Singh Juggut,India,India,Mano Majra,Mano Majra
1,Train to Pakistan,12.01,Nooran Unknown,India,India,Mano Majra,Mano Majra
14,Tamas,27.01,Deputy Commissioner Richard,Pakistan,Pakistan,Hindu Kush,Rawalpindi
15,Tamas,27.01,Richard's wife Liza,Pakistan,Pakistan,Hindu Kush,Hindu Kush
25,Sunlight on a Broken Column,156.01,Viceroy Unknown,India,India,Lucknow University,Lucknow University
...,...,...,...,...,...,...,...
5087,A Life Long Ago,62.03,bhai Acchar,Bangladesh,Bangladesh,Digpait,Digpait
5089,The Price of Freedom,45.01,Ali Shahzada Ghulam,India,India,Bombay,Bombay
5090,The Price of Freedom,47.01,Wife of Shahzada Ghulam Ali Nigar,India,India,Amritsar,Bombay
5091,The Price of Freedom,51.01,Young girl Unnamed,India,India,Jallianwala Bagh,Jallianwala Bagh


In [169]:
data['has_changed'] = data['initial_country'] != data['location_country']

In [165]:
data['has_immigrated'] = data['initial_country'] != data['final_country']

In [166]:
print(len(data[data['has_immigrated'] == False]))
print(len(data[data['has_immigrated'] == True]))

2072
368


In [167]:
data.dtypes

text_title                  object
event_id                     int64
start_page_event             int64
page_event_number            int64
text_place_title            object
character_id               float64
family_name                 object
given_name                  object
page_first_mentioned       float64
number                      object
religion                    object
relig_att_char              object
country_of_birth            object
place_name                  object
location_religion           object
relig_att_loc               object
geolocation                 object
location_name               object
location_country            object
author_birth_location       object
country_of_birth_author     object
text_place_id                int64
location_id                float64
author_id                    int64
author_birth_place_id        int64
full_name                   object
event_order                float64
initial_country             object
final_country       